# Interpolation Tools Quality Check

The code below will use the interpolation functions for data sets that relate $\% H_2SO_4$ to $H_0$, $H_R$, $a_{H_2O}$, and density and plot the iterpolation function vs the actual data. Hopefully the two will line up.

In [1]:
##############################################################
### Set up libraries and global variables
##############################################################

H0_Data_File_Name = "02_H0_H2SO4_Tickle_1970.csv"
HR_Data_File_Name = "04_Deno_1955_HR_Values.csv"
aH2O_Data_File_Name = "01_Giauque_Table9_activityH2OinH2SO4.csv"
Density_Data_File_Name = "03_H2SO4_PercentWt_to_MolarConc_CRC.csv"
#Density_Data_File_Name = "03_H2SO4_PercentWt_to_MolarConc.csv"

Filenames_Dictionary = {"H0":H0_Data_File_Name,
                         "HR":HR_Data_File_Name,
                         "aH2O":aH2O_Data_File_Name,
                         "Density":Density_Data_File_Name
                         }

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import interpolators            # The external library with the interpolators.

#github_location = "https://raw.githubusercontent.com/blinkletter/4410PythonNotebooks/main/Class_30/data/"
github_location = "./data/"
github_H0_location = "https://raw.githubusercontent.com/blinkletter/4410PythonNotebooks/main/Class_08/data/H0_data/"
github_location_styles = "https://raw.githubusercontent.com/blinkletter/LFER-QSAR/main/styles/"
github_location_LFER_tables = "https://raw.githubusercontent.com/blinkletter/LFER-QSAR/main/data/"



In [3]:
##############################################################
### Import the data and interpolator
##############################################################

Parameter = "aH2O"    ### Use "H0" or "HR" or "aH2O" or "Density"

Data_File_Name = Filenames_Dictionary[Parameter]   
Filename = github_location + Data_File_Name

df = pd.read_csv(Filename, 
                 delimiter = ",", 
                 skipinitialspace=True, 
#                 index_col="Substituent", 
                 comment = "#") 

interpolator = interpolators.select_interpolator(Parameter)

##############################################################
### Select data
##############################################################

x = df["%H2SO4"]
y = df[Parameter]

step = 0.1
xi = np.arange(np.min(x), np.max(x)+step, step)
yi = interpolator(xi)

##############################################################
### Plot Data vs. Interpolation
##############################################################

print(Parameter)
plt.plot(x,y,".")
plt.plot(xi,yi,"-")
plt.show()

##############################################################
### Plot Diff Between Data and Interpolation
##############################################################

plt.plot(x,y-interpolator(x),"-")
plt.show()



FileNotFoundError: [Errno 2] No such file or directory: './data/01_Giauque_Table9_activityH2OinH2SO4_CRC.csv'